In [1]:
import sys
sys.path.append('../../hw/base') # just for the boilerplate stuff
sys.path.append('../../hw/cpu') 

from verification_utils import CHCs, mk_int_array
import boilerplate
from instruction_set import *
from assembler import asm, disasm, disasm_pretty, asm_bin


In [2]:
from z3 import *
import z3
z3.set_param(proof=True)  # this is needed so that we can call `.proof()` later

In [3]:
stack = Array('stack', IntSort(), IntSort())
sp = Int('sp')
r0 = Int('r0')
r1 = Int('r1')

state_vars = [stack, sp]


a, b = input_vars = Ints('a b')

In [4]:
"""
{  stack= [a, b] ∧ b > 0 }
  DUP 1
  PUSH 0
loop:
  DUP; DUP 3; POP 2; ALU LT; POP 1; JZ exit
  DUP 1; PUSH 1; POP 2; ALU ADD
  DUP 1; PUSH 1; POP 2; ALU ADD
  YANK 2,2
  JMP loop
exit:
  POP 1
{ stack = [a, b, a + b] }
"""

# a + b

instrList= [('DUP', 1), ('PUSH', 0), 'loop', ('DUP', 0), ('DUP', 3), ('POP', 2), ('ALU', 'LT'), ('POP', 1), ('JZ', 'exit'),
            ('DUP', 1), ('PUSH', 1), ('POP', 2), ('ALU', 'ADD'), ('DUP', 1), ('PUSH', 1), ('POP', 2), ('ALU', 'ADD'), ('YANK', (2, 2)), ('JMP', 'loop'), 'exit', ('POP', 1)]

asm_list = asm(instrList, start_addr=0)
dict= asm_list.label_addrs

sigma = [*state_vars]
U = {i: Function(f"U{i}", *(v.sort() for v in sigma), BoolSort()) for i in range(len(instrList) + 1)}

# init state of stack and sp, stack[a, b]
q1 = stack[sp - 1] > 0
# final state of the vars
q2 = stack[sp - 1] == stack[sp - 3] + stack[sp - 2]



In [5]:
instrList = [ 'div_mod_3', ('DUP', 0), ('PUSH', 0), 'div_mod_aux', ('DUP', 1), ('PUSH', 3), ('POP', 2), ('ALU', 'LT'),
 ('POP', 1), ('JZ', '_else_0'), ('JMP', '_end_0'), '_else_0', ('DUP', 1), ('PUSH', 3), 
 ('POP', 2), ('ALU', 'SUB'), ('DUP', 1), ('PUSH', 1), ('POP', 2), ('ALU', 'ADD'), ('YANK', (2, 2)), ('JMP', 'div_mod_aux'), '_end_0']

# div_mod (THIS IS DIV and MOD) general with a const denominator 
'''
div_mod_3(1) = div_mod_aux $0 $0 0
div_mod_aux(3) = if $1 < 3 then $2
                           else .div_mod_aux $0 ($1 - 3) ($2 + 1)


div_mod_3: 
    DUP 0
    PUSH 0    
div_mod_aux (loop):
    DUP 1; PUSH 3; POP 2; ALU LT; POP 1; JZ _else_0
    JMP _end_0
_else_0:
    DUP 1; PUSH 3; POP 2; ALU SUB
    DUP 1; PUSH 1; POP 2; ALU ADD
    YANK 2 2
    JMP div_mod_aux (loop)
_end_0:

'''

asm_list = asm(instrList, start_addr=0)
dict= asm_list.label_addrs

sigma = [*state_vars]
U = {i: Function(f"U{i}", *(v.sort() for v in sigma), BoolSort()) for i in range(len(instrList) + 1)}

# init state of stack and sp, stack = [b] and b > 0
q1 = stack[sp - 1] > 0
# final state of the vars, stack = [b, mod, div], b = 3*div + mod
# unsat because sat is taking too long to load!
q2 = stack[sp - 3] == (3*stack[sp - 1] + stack[sp - 2])

In [6]:
instrList = ['checkHeadUp', ('DUP', 0), ('PUSH', 40960), ('POP', 2), ('ALU', 'LT'), ('POP', 1), ('JZ', '_else_0'), ('PUSH', 1), ('JMP', '_end_0'),
             '_else_0', ('PUSH', 40975), ('DUP', 1), ('POP', 2), ('ALU', 'LT'), ('POP', 1), ('JZ', '_else_1'), ('PUSH', 1),
             ('JMP', '_end_1'), '_else_1', ('PUSH', 0), '_end_1', '_end_0']

# checkHeadUp from snake
'''
0xa000 <= $0 <= 0xa00f --> return 0 else return 1

checkHeadUp(1) = if $0 < 0xa000 then 1 else (if 0xa00f < $0 then 1 else 0)
'''

asm_list = asm(instrList, start_addr=0)
dict= asm_list.label_addrs

sigma = [*state_vars]
U = {i: Function(f"U{i}", *(v.sort() for v in sigma), BoolSort()) for i in range(len(instrList) + 1)}

# init state of stack and sp, stack[a]
q1 = And(stack[sp - 1] >= 0xa000, stack[sp - 1] <= 0xa00f) 
# final state of the vars
q2 = stack[sp - 1] == 0

In [8]:
instrList = ['find_index_val', ('DUP', 2), ('DUP', 2), ('DUP', 2), ('PUSH', 0), 'find_index_val_aux', ('DUP', 0), ('DUP', 3), ('POP', 2), ('ALU', 'LT'), ('POP', 1),
             ('JZ', '_else_0'), ('DUP', 3), ('DUP', 1), ('POP', 2), ('ALU', 'ADD'), ('POP', 1), ('LOAD', 0), ('DUP', 2),  ('POP', 2), ('ALU', 'SUB'), ('POP', 1),
             ('JZ', '_else_1'), ('PUSH', 1), ('POP', 2), ('ALU', 'ADD'), ('JMP', 'find_index_val_aux'), '_else_1', ('DUP', 0), ('JMP', '_end_1'), '_else_0', ('DUP', 2),
             '_end_1', ('YANK', (1, 4))]
# find index of val
'''
find_index_val(3) = find_index_val_aux $0 $1 $2 0
find_index_val_aux(4) = if $3 < $1 then (if (mem_peek ($0 + $3) - $2) then .find_index_val_aux $0 $1 $2 ($3 + 1) else $3) else $1


find_index_val:
    DUP 2
    DUP 2
    DUP 2
    PUSH 0
find_index_val_aux:
    DUP 0
    DUP 3
    POP 2; ALU LT; POP 1; JZ _else_0
    DUP 3 
    DUP 1
    POP 2; ALU ADD
    POP 1; LOAD
    DUP 2
    POP 2; ALU SUB; POP 1; JZ _else_1
    PUSH 1
    POP 2
    ALU ADD 
    JMP find_index_val_aux
_else_1:
    DUP 0
    JMP _end_1
_else_0:
    DUP 2
_end_1:
    YANK 1 4
    
'''


asm_list = asm(instrList, start_addr=0)
dict= asm_list.label_addrs

sigma = [*state_vars]
U = {i: Function(f"U{i}", *(v.sort() for v in sigma), BoolSort()) for i in range(len(instrList) + 1)}

# init state of stack and sp, stack[init_adrr, array_length], init_adrr > 0, array_length > 0
'''
stack = [j, init_address, length, val], init_address >= 0, j >= 0 and j < length, stack[init_adress + j] == val,  length < 6
'''
q1 = And(stack[sp - 4] >= 0, stack[sp - 4] < stack[sp - 2], stack[stack[sp - 3] + stack[sp - 4]] == stack[sp - 1], stack[sp - 2] < 6, sp > (stack[sp - 2] + stack[sp - 3] + 4))
# final state of the vars
'''
stack = [j, init_address, length, val, returned_index], returned_index <= j
'''
q2 = stack[sp - 1] <= stack[sp - 5]

In [9]:
instrList = ['find_max', ('DUP', 1), ('DUP', 1), ('PUSH', 1), ('DUP', 4), ('POP', 1), ('LOAD', 0), 'find_max_aux', ('DUP', 1), ('DUP', 3), ('POP', 2),
             ('ALU', 'LT'), ('POP', 1), ('JZ', '_else_0'), ('DUP', 0), ('DUP', 4), ('DUP', 3), ('POP', 2), ('ALU', 'ADD'), ('POP', 1), ('LOAD', 0),
             ('POP', 2), ('ALU', 'LT'), ('POP', 1), ('JZ', '_else_1'), ('DUP', 3), ('DUP', 3), ('DUP', 3), ('PUSH', 1), ('POP', 2), ('ALU', 'ADD'), 
            ('DUP', 6), ('DUP', 5), ('POP', 2), ('ALU', 'ADD'), ('POP', 1), ('LOAD', 0), ('YANK', (4, 4)), ('JMP', 'find_max_aux'),
             '_else_1', ('DUP', 3), ('DUP', 3), ('DUP', 3), ('PUSH', 1), ('POP', 2), ('ALU', 'ADD'), ('DUP', 3), ('YANK', (4, 4)), ('JMP', 'find_max_aux'),
              '_else_0', ('DUP', 0), '_end_1', ('YANK', (1, 4))]

# find max in array
'''
    find_max(2) = find_max_aux $0 $1 1 (mem_peek $0)
    find_max_aux(4) = if $2 < $1 then (if $3 < mem_peek ($0 + $2) then .find_max_aux $0 $1 ($2 + 1) (mem_peek ($0 + $2)) else .find_max_aux $0 $1 ($2 + 1) $3
                                ) else $3

find_max:
    DUP 1
    DUP 1
    PUSH 1
    DUP 4
    POP 1; LOAD
find_max_aux:
    DUP 1
    DUP 3
    POP 2; ALU LT; POP 1; JZ _else_0
    DUP 0
    DUP 4
    DUP 3
    POP 2; ALU ADD
    POP 1; LOAD
    POP 2; ALU LT; POP 1; JZ _else_1
    DUP 3
    DUP 3
    DUP 3
    PUSH 1
    POP 2; ALU ADD
    DUP 6
    DUP 5
    POP 2; ALU ADD
    POP 1; LOAD
    YANK 4 4
    JMP find_max_aux
_else_1:
    DUP 3
    DUP 3
    DUP 3
    PUSH 1
    POP 2; ALU ADD
    DUP 3
    YANK 4 4
    JMP find_max_aux
_else_0:
    DUP 0
_end_1:
    YANK 1 4
    
'''


asm_list = asm(instrList, start_addr=0)
dict= asm_list.label_addrs

sigma = [*state_vars]
U = {i: Function(f"U{i}", *(v.sort() for v in sigma), BoolSort()) for i in range(len(instrList) + 1)}

# init state of stack and sp, stack[init_adrr, array_length], init_adrr > 0, array_length > 0
'''
stack = [j, init_address_array, array_length] and 0 <= j < array_length and init_address_array >= 15
'''
q1 = And(stack[sp - 3] >= 0, stack[sp - 3] < stack[sp - 1], stack[sp - 1] < 6, sp > (stack[sp - 1] + stack[sp - 2] + 3))
# final state of the vars
'''
stack = [j, init_address_array, array_length, max_value] , stack[init_address_array + j] <= max_value
'''
q2 = stack[stack[sp - 3] + stack[sp - 4]] <= stack[sp - 1]

In [10]:
    sigma = [*state_vars]
    U = {i: Function(f"U{i}", *(v.sort() for v in sigma), BoolSort()) for i in range(len(instrList) + 1)}
    chcs_to_solve = [Implies(q1, U[0](sigma))]
    addresses_list = []
    len_list = len(instrList)
    i = 0
    while (i < len_list):
        #print(instrList[i])
        #print(type(instrList[i]))
        #print(i)
        if isinstance(instrList[i], str):
            to_add = Implies(U[i](sigma), U[i+1](sigma))
            chcs_to_solve.append(to_add)
            i = i + 1
        else:
            instr, inp = instrList[i]
            if instr == 'PUSH':
                if isinstance(inp, str):
                    to_add = Implies(U[i](sigma), U[i+1](Store(stack, sp, instrList.index(inp)), sp + 1))
                    addresses_list.append(instrList.index(inp))
                else: to_add = Implies(U[i](sigma), U[i+1](Store(stack, sp, inp), sp + 1))
                chcs_to_solve.append(to_add)
                i = i + 1
                continue
            elif instr == 'POP':
                if inp == 1:
                    if (i + 1) < len_list:
                        instr, inp = instrList[i + 1]
                        if instr == 'ALU':
                            if inp == 'NEG':
                                result = -stack[sp - 1]
                            elif inp == 'NOT':
                                result = ~stack[sp - 1] 
                   
                            to_add = Implies(U[i](sigma), U[i+2](Store(stack, sp - 1, result), sp))
                            chcs_to_solve.append(to_add)
                            i = i + 2
                            continue
                        elif instr == 'LOAD':
                            to_add = Implies(U[i](sigma), U[i+2](Store(stack, sp - 1, stack[stack[sp - 1]]), sp))
                            chcs_to_solve.append(to_add)
                            i = i + 2
                            continue  
                        else:
                            to_add = Implies(U[i](sigma), U[i+1](stack, sp - 1))
                            chcs_to_solve.append(to_add)
                            i = i + 1
                            continue                     
                    else:
                        to_add = Implies(U[i](sigma), U[i+1](stack, sp - 1))
                        chcs_to_solve.append(to_add)
                        i = i + 1
                        continue
                elif inp == 2:                   
                    if (i + 1) < len_list:
                        instr, inp = instrList[i + 1]
                        if instr == 'ALU':
                            if inp == 'ADD':
                                result = stack[sp - 1] + stack[sp - 2]
                            elif inp == 'SUB':
                                result = stack[sp - 2] - stack[sp - 1] 
                                if (i + 2) < len_list and (i + 3) < len_list:
                                    instr, inp = instrList[i + 2]
                                    if instr == 'POP' and inp == 1:
                                        instr, inp = instrList[i + 3]
                                        if instr == 'JZ':
                                            result_true = result != 0
                                            result_false = result == 0
                                            to_add = Implies(And(U[i](sigma), result_true), U[i+4](stack, sp - 2))
                                            chcs_to_solve.append(to_add)
                                            to_add = Implies(And(U[i](sigma), result_false), U[instrList.index(inp) + 1](stack, sp - 2))
                                            chcs_to_solve.append(to_add)
                                            i = i + 4
                                            continue    
                                        elif instr == 'JNZ':
                                            to_add = Implies(And(U[i](sigma), result_false), U[i+4](stack, sp - 2))
                                            chcs_to_solve.append(to_add)
                                            to_add = Implies(And(U[i](sigma), result_true), U[instrList.index(inp) + 1](stack, sp - 2))
                                            chcs_to_solve.append(to_add)
                                            i = i + 4
                                            continue 
                            elif inp == 'MUL':
                                result = stack[sp - 1] * stack[sp - 2]
                            elif inp == 'SHL':
                                result = stack[sp - 2] << stack[sp - 1]
                            elif inp == 'SHR':
                                result = stack[sp - 2] >> stack[sp - 1]
                            elif inp == 'AND':
                                result = stack[sp - 2] & stack[sp - 1]
                            elif inp == 'OR':
                                result = stack[sp - 2] | stack[sp - 1]
                            elif inp == 'LT':                                
                                result_true = stack[sp - 2] < stack[sp - 1]
                                result_false = stack[sp - 2] >= stack[sp - 1]
                                if (i + 2) < len_list and (i + 3) < len_list:
                                    instr, inp = instrList[i + 2]
                                    if instr == 'POP' and inp == 1:
                                        instr, inp = instrList[i + 3]
                                        if instr == 'JZ':
                                            # jump when result_false
                                            #print("i am here")
                                            to_add = Implies(And(U[i](sigma), result_true), U[i+4](stack, sp - 2))
                                            chcs_to_solve.append(to_add)
                                            to_add = Implies(And(U[i](sigma), result_false), U[instrList.index(inp) + 1](stack, sp - 2))
                                            chcs_to_solve.append(to_add)
                                            i = i + 4
                                            continue    
                                        elif instr == 'JNZ':
                                            to_add = Implies(And(U[i](sigma), result_false), U[i+4](stack, sp - 2))
                                            chcs_to_solve.append(to_add)
                                            to_add = Implies(And(U[i](sigma), result_true), U[instrList.index(inp) + 1](stack, sp - 2))
                                            chcs_to_solve.append(to_add)
                                            i = i + 4
                                            continue 
                            
                            to_add = Implies(U[i](sigma), U[i+2](Store(stack, sp - 2, result), sp - 1))
                            chcs_to_solve.append(to_add)
                            i = i + 2
                            continue
                        elif instr == 'STOR':
                            to_add = Implies(U[i](sigma), U[i+2](Store(stack, stack[sp - 1], stack[sp - 2]), sp - 2))
                            chcs_to_solve.append(to_add)
                            i = i + 2
                            continue  
                        else:
                            to_add = Implies(U[i](sigma), U[i+1](stack, sp - 2))
                            chcs_to_solve.append(to_add)
                            i = i + 1
                            continue                     
                    else:
                        to_add = Implies(U[i](sigma), U[i+1](stack, sp - 2))
                        chcs_to_solve.append(to_add)
                        i = i + 1
                        continue                    
            elif instr == 'DUP':
                to_add = Implies(U[i](sigma), U[i+1](Store(stack, sp, stack[sp - inp - 1]), sp + 1))
                chcs_to_solve.append(to_add)
                i = i + 1
                continue                  
            elif instr == 'YANK':
                tmp = stack
                x, y = inp
                k = 0
                j = y
                while k < y:
                    tmp = Store(tmp, sp - x - j, stack[sp - x + k])
                    k = k + 1
                    j = j - 1
                to_add = Implies(U[i](sigma), U[i+1](tmp, sp - y))
                chcs_to_solve.append(to_add)
                i = i + 1
                continue   
            elif instr == 'JMP':
                to_add = Implies(U[i](sigma), U[instrList.index(inp) + 1](sigma))
                chcs_to_solve.append(to_add)
                i = i + 1
                continue             
    chcs_to_solve.append(Implies(U[len(instrList)](sigma), q2)) 
    chcs = CHCs(chcs_to_solve)
    chcs

"stack[sp - 3] ≥ 0 ∧ stack[sp - 3] < stack[sp - 1] ∧ stack[sp - 1] < 6 ∧ sp > stack[sp - 1] + stack[sp - 2] + 3 ⇒ U0(stack, sp)"
"U0(stack, sp) ⇒ U1(stack, sp)"
"U1(stack, sp) ⇒ U2(Store(stack, sp, stack[sp - 1 - 1]), sp + 1)"
"U2(stack, sp) ⇒ U3(Store(stack, sp, stack[sp - 1 - 1]), sp + 1)"
"U3(stack, sp) ⇒ U4(Store(stack, sp, 1), sp + 1)"
"U4(stack, sp) ⇒ U5(Store(stack, sp, stack[sp - 4 - 1]), sp + 1)"
"U5(stack, sp) ⇒ U7(Store(stack, sp - 1, stack[stack[sp - 1]]), sp)"
"U7(stack, sp) ⇒ U8(stack, sp)"
"U8(stack, sp) ⇒ U9(Store(stack, sp, stack[sp - 1 - 1]), sp + 1)"
"U9(stack, sp) ⇒ U10(Store(stack, sp, stack[sp - 3 - 1]), sp + 1)"
"U10(stack, sp) ∧ stack[sp - 2] < stack[sp - 1] ⇒ U14(stack, sp - 2)"


In [11]:
s = chcs.create_solver()
#s.set('xform.inline_eager', False)
#s.set('xform.inline_linear', False)
#s.set(timeout=1000)
%time s.check()

CPU times: user 133 ms, sys: 650 µs, total: 134 ms
Wall time: 145 ms


unsat